In [9]:
import sqlite3
import pandas as pd
import numpy as np
import os

#  Giai đoạn EXTRACT
def extract_data_from_db(db_path):
    print("Đang tải dữ liệu từ Database...")
    conn = sqlite3.connect(db_path)
   
    try:
        df_sales = pd.read_sql_query("SELECT product_id, all_time_quantity_sold, crawl_timestamp FROM sales_history", conn)
        df_price = pd.read_sql_query("SELECT product_id, price, crawl_timestamp FROM price_history", conn)
        df_rating = pd.read_sql_query("SELECT product_id, review_count, rating_average, crawl_timestamp FROM rating_history", conn)
        df_info = pd.read_sql_query("SELECT id as product_id, name, created_at FROM products", conn)
       
        print(f" -> Đã load thành công dữ liệu từ {db_path}")
        return df_sales, df_price, df_rating, df_info
   
    except Exception as e:
        print(f"LỖI khi đọc DB: {e}")
        return None, None, None, None
    finally:
        conn.close()

# Giai đoạn TRANSFORM
def transform_data(df_sales, df_price, df_rating, df_info):
    print("Đang xử lý và tính toán các chỉ số (Feature Engineering)...")
   
    # Ép kiểu datetime
    for df in [df_sales, df_price, df_rating]:
        df['crawl_timestamp'] = pd.to_datetime(df['crawl_timestamp'])
    min_date = df_sales['crawl_timestamp'].min()
    max_date = df_sales['crawl_timestamp'].max()
    print(f" -> Dữ liệu trải dài từ: {min_date} đến {max_date}")
    first_day = min_date.date()

    # Lưu rating_average cuối cùng
    df_rating_avg = df_rating.sort_values('crawl_timestamp').drop_duplicates('product_id', keep='last')[['product_id', 'rating_average']]

    # Aggregate last per day
    def aggregate_daily_last(df, val_col):
        df['date'] = df['crawl_timestamp'].dt.floor('D') 
        df_daily = df.sort_values('crawl_timestamp').groupby(['product_id', 'date'])[val_col].last().reset_index()
        df_daily['crawl_timestamp'] = pd.to_datetime(df_daily['date'])
        return df_daily
   
    df_sales = aggregate_daily_last(df_sales, 'all_time_quantity_sold')
    df_price = aggregate_daily_last(df_price, 'price')
    df_rating = aggregate_daily_last(df_rating, 'review_count')

    # Tính daily & weekly sales
    print(" -> Đang tính số lượng bán theo ngày và theo tuần...")
    df_sales_daily = df_sales.copy()
    df_sales_daily = df_sales_daily.sort_values(['product_id', 'crawl_timestamp'])
    df_sales_daily['daily_quantity_sold'] = df_sales_daily.groupby('product_id')['all_time_quantity_sold'].diff()
    df_sales_daily['daily_quantity_sold'] = df_sales_daily['daily_quantity_sold'].fillna(0).clip(lower=0)

    df_sales_daily['week_start'] = df_sales_daily['date'] - pd.to_timedelta(df_sales_daily['date'].dt.weekday, unit='D')

    df_sales_weekly = df_sales_daily.groupby(['product_id', 'week_start'])['daily_quantity_sold'].sum().reset_index()
    df_sales_weekly.rename(columns={'daily_quantity_sold': 'weekly_quantity_sold'}, inplace=True)

    # Hàm lấy giá trị đầu/kỳ cuối 
    def get_period_values(df, val_col, rename_col_new, rename_col_old):
        df_new = df.sort_values('crawl_timestamp').drop_duplicates('product_id', keep='last')[['product_id', val_col]]
        df_new.rename(columns={val_col: rename_col_new}, inplace=True)
        df_old_temp = df[df['date'] == first_day]
        if not df_old_temp.empty:
            df_old = df_old_temp.sort_values('crawl_timestamp').drop_duplicates('product_id', keep='last')[['product_id', val_col]]
        else:
            df_old = df.sort_values('crawl_timestamp').drop_duplicates('product_id', keep='first')[['product_id', val_col]]
        df_old.rename(columns={val_col: rename_col_old}, inplace=True)
        return df_new, df_old

    df_sales_new, df_sales_old = get_period_values(df_sales, 'all_time_quantity_sold', 'qty_new', 'qty_old')
    df_price_new, df_price_old = get_period_values(df_price, 'price', 'price_new', 'price_old')
    df_review_new, df_review_old = get_period_values(df_rating, 'review_count', 'review_new', 'review_old')

    # Merge tất cả vào df_final
    df_final = df_info.drop_duplicates(subset=['product_id'], keep='last').copy()
    for df_merge in [df_sales_new, df_sales_old, df_price_new, df_price_old,
                     df_review_new, df_review_old, df_rating_avg]:
        df_final = pd.merge(df_final, df_merge, on='product_id', how='left')

    # Xử lý missing
    df_final['qty_old'] = df_final['qty_old'].fillna(0)
    df_final['qty_new'] = df_final['qty_new'].fillna(df_final['qty_old'])
    df_final['price_old'] = df_final['price_old'].fillna(df_final['price_new'])
    df_final['review_old'] = df_final['review_old'].fillna(df_final['review_new'])
    df_final['review_new'] = df_final['review_new'].fillna(df_final['review_old'])

    # Tính các chỉ số
    # 1. quantity_sold_in_period
    df_final['quantity_sold_in_period'] = (df_final['qty_new'] - df_final['qty_old']).clip(lower=0)
    # 2. price_change_rate
    df_final['price_change_raw'] = np.where(
        df_final['price_old'] > 0,
        (df_final['price_new'] - df_final['price_old']) / df_final['price_old'],
        0
    )
    df_final['price_change_rate'] = (df_final['price_change_raw'] * 100).round(2)
    

    # days_on_shelf 
    stats = df_sales.groupby('product_id')['date'].agg(['min', 'max'])
    stats['days_on_shelf'] = (stats['max'] - stats['min']).dt.days + 1  # ← ĐÃ FIX: dùng .dt.days, +1 để tính đúng
    stats = stats[['days_on_shelf']].reset_index()
    df_final = pd.merge(df_final, stats, on='product_id', how='left')

    # 3. review_growth_rate
    df_final['review_growth'] = df_final['review_new'] - df_final['review_old']
    df_final['review_growth_rate'] = (df_final['review_growth'] / df_final['days_on_shelf']).clip(lower=0).round(2)

    qty_change_rate = np.where(df_final['qty_old'] > 0,
                               df_final['quantity_sold_in_period'] / df_final['qty_old'], 0)
    #4. price_elasticity
    df_final['price_elasticity'] = np.where(
        (df_final['price_change_raw'] != 0) & (df_final['qty_old'] > 0),
        qty_change_rate / df_final['price_change_raw'], 0
    )
    df_final['price_elasticity'] = df_final['price_elasticity'].replace([np.inf, -np.inf], 0).round(2)

    

    df_final['sales_per_day'] = (df_final['quantity_sold_in_period'] / df_final['days_on_shelf']).round(2)

    return df_final, df_sales_daily, df_sales_weekly

# Giai đoạn LOAD
def save_data_to_csv(df_final, df_daily, df_weekly, output_dir):
    print("Đang lưu file kết quả...")
    os.makedirs(output_dir, exist_ok=True)

    target_cols = ['product_id', 'name', 'quantity_sold_in_period']
    df_final[target_cols].to_csv(f'{output_dir}/target_variable.csv', index=False)
    print(f"Đã xuất file 1: target_variable.csv")

    cols_view = [
        'product_id', 'name', 'created_at',
        'days_on_shelf', 'qty_old', 'qty_new', 'quantity_sold_in_period',
        'sales_per_day', 'price_old', 'price_new', 'price_change_rate',
        'price_elasticity', 'review_new', 'review_growth_rate', 'rating_average'
    ]
    valid_cols = [c for c in cols_view if c in df_final.columns]
    df_final[valid_cols].to_csv(f'{output_dir}/feature_analysis_full.csv', index=False)
    print(f"-> Đã lưu: {output_dir}/feature_analysis_full.csv (tổng hợp sản phẩm)")

    df_daily = pd.merge(df_daily, df_final[['product_id', 'name']], on='product_id', how='left')
    df_weekly = pd.merge(df_weekly, df_final[['product_id', 'name']], on='product_id', how='left')

    df_daily[['product_id', 'name', 'date', 'daily_quantity_sold']].to_csv(
        f'{output_dir}/daily_sales.csv', index=False)
    print(f"-> Đã lưu: {output_dir}/daily_sales.csv (chi tiết theo ngày)")

    df_weekly[['product_id', 'name', 'week_start', 'weekly_quantity_sold']].to_csv(
        f'{output_dir}/weekly_sales.csv', index=False)
    print(f"-> Đã lưu: {output_dir}/weekly_sales.csv (tổng theo tuần)")

    # Tạo long format đúng chuẩn
    df_long = df_daily[['product_id', 'date', 'daily_quantity_sold']].copy()

    # Tạo cột week_start từ ngày (để merge với weekly)
    df_long['week_start'] = df_long['date'] - pd.to_timedelta(df_long['date'].dt.weekday, unit='D')

    # Merge thông tin tĩnh của sản phẩm (name, days_on_shelf, sales_per_day, giá, elasticity, rating, v.v.)
    df_long = df_long.merge(
        df_final[['product_id', 'name', 'created_at','days_on_shelf', 'quantity_sold_in_period', 
                  'sales_per_day','price_old', 'price_new', 'price_change_rate',
                  'price_elasticity', 'review_new', 'review_growth_rate', 'rating_average']],
        on='product_id',
        how='left'
    )

    # Merge doanh số tuần (match đúng theo tuần)
    df_long = df_long.merge(
        df_weekly[['product_id', 'week_start', 'weekly_quantity_sold']],
        on=['product_id', 'week_start'],
        how='left'
    )

    # Sắp xếp đẹp
    df_long = df_long.sort_values(['product_id', 'date']).reset_index(drop=True)

    # Lưu file
    df_long.to_csv(f'{output_dir}/full_analysis_with_daily_weekly.csv', index=False)
    print(f"-> Đã lưu file long format: full_analysis_with_daily_weekly.csv")
    print(f"   Kích thước: {len(df_long):,} dòng × {len(df_long.columns)} cột")

    print("-" * 30)
    print("Top 10 sản phẩm bán chạy nhất:")
    display(df_final[valid_cols].sort_values(by='sales_per_day', ascending=False).head(10))
   

def main():
    DB_PATH = '../data/database/tiki_products_multi.db'
    OUTPUT_DIR = '../data/processed'

    if not os.path.exists(DB_PATH):
        print(f"Không tìm thấy file DB tại: {DB_PATH}")
        return

    sales, price, rating, info = extract_data_from_db(DB_PATH)
   
    if sales is not None:
        final_df, daily_df, weekly_df = transform_data(sales, price, rating, info)
        save_data_to_csv(final_df, daily_df, weekly_df, OUTPUT_DIR)

if __name__ == "__main__":
    main()

Đang tải dữ liệu từ Database...
 -> Đã load thành công dữ liệu từ ../data/database/tiki_products_multi.db
Đang xử lý và tính toán các chỉ số (Feature Engineering)...
 -> Dữ liệu trải dài từ: 2025-11-02 10:45:20.297907 đến 2025-11-23 22:21:23.833893
 -> Đang tính số lượng bán theo ngày và theo tuần...
Đang lưu file kết quả...
Đã xuất file 1: target_variable.csv
-> Đã lưu: ../data/processed/feature_analysis_full.csv (tổng hợp sản phẩm)
-> Đã lưu: ../data/processed/daily_sales.csv (chi tiết theo ngày)
-> Đã lưu: ../data/processed/weekly_sales.csv (tổng theo tuần)
-> Đã lưu file long format: full_analysis_with_daily_weekly.csv
   Kích thước: 83,071 dòng × 17 cột
------------------------------
Top 10 sản phẩm bán chạy nhất:


,product_id,name,created_at,days_on_shelf,qty_old,qty_new,quantity_sold_in_period,sales_per_day,price_old,price_new,price_change_rate,price_elasticity,review_new,review_growth_rate,rating_average
79,767101,Bao cao su Durex Kingtex 12 bao,2025-11-23 16:38:18,22,11851,70895,59044,2683.82,131000.0,117000.0,-10.69,-46.62,1472,0.36,4.6
8,385582,Bao cao su Durex Fetherlite Hộp 12 Bao,2025-11-23 16:38:18,21,3783,29837,26054,1240.67,203000.0,195000.0,-3.94,-174.76,219,0.00,4.7
53,543129,Nước Giặt Quần Áo Cho Bé D-nee - Chai 3000ml (...,2025-11-23 16:38:18,22,206,26198,25992,1181.45,190000.0,229000.0,20.53,614.70,415,17.95,4.5
673,53583472,Bộ Hộp Cơm Giữ Nhiệt Lock&Lock Easy Carry 2L L...,2025-11-23 16:38:18,15,572,11934,11362,757.47,1037900.0,1216000.0,17.16,115.76,2718,172.93,4.7
430,24028050,Sữa bột Abbott Pediasure 1.6kg cho trẻ từ 1-10...,2025-11-23 16:38:18,21,894,15122,14228,677.52,1309000.0,1263000.0,-3.51,-452.89,824,31.10,4.8
439,25108217,Sữa rửa mặt ngăn ngừa mụn Acnes Creamy Wash 100g,2025-11-23 16:38:18,22,173170,186252,13082,594.64,59000.0,59000.0,0.00,0.00,239,0.05,5.0
21,414764,Lò Vi Sóng Sharp R-205VN(S) - 20L - Hàng chính...,2025-11-23 16:38:18,21,210,10633,10423,496.33,1770000.0,1526000.0,-13.79,-360.05,1209,54.57,4.7
501,34537693,Tã Dán Siêu Thấm Caryn Gói Siêu Tiết Kiệm L40 ...,2025-11-23 16:38:18,22,26,9394,9368,425.82,440000.0,399000.0,-9.32,-3866.72,695,31.55,5.0
218,6874429,Miếng dán mụn giúp giảm mụn sưng viêm Acnes Cl...,2025-11-23 16:38:18,22,116679,124610,7931,360.50,50500.0,50500.0,0.00,0.00,323,0.05,4.7
3428,276342461,Kem Dưỡng Ẩm Da NIVEA Crème (30 ml) - 80101,2025-11-23 16:38:18,22,29938,37568,7630,346.82,51000.0,51000.0,0.00,0.00,1,0.00,5.0
